# Customer Deduplication with RowVoi

## Business Problem

Your company has a customer database that has grown over time through multiple sources:
- Website registrations
- Trade show sign-ups  
- Sales team entries
- Partner referrals

**The Problem:** You suspect many customers appear multiple times with slight variations in their information. This leads to:
- ❌ Duplicate marketing emails (annoying customers)
- ❌ Inflated customer counts (wrong metrics)
- ❌ Wasted marketing budget
- ❌ Poor customer experience

**The Solution:** Use RowVoi to intelligently identify which customer fields to check to find and resolve duplicates efficiently.

## What You'll Learn

1. 🎯 **Find minimal distinguishing fields** - Which customer attributes actually matter for deduplication?
2. 🔍 **Interactive disambiguation** - How to resolve ambiguous cases step-by-step
3. 💰 **Cost-aware selection** - Balance accuracy vs. effort when some fields are expensive to verify
4. 📊 **Measure success** - Quantify the impact of your deduplication strategy

## 1. Load and Explore the Data

In [ ]:
import pandas as pd

from rowvoi import (
    DisambiguationSession,
    GreedyCoveragePolicy,
    find_key,
)

# Load our customer database
df = pd.read_csv('../data/customers_sample.csv')
print(f"📊 Customer Database: {len(df)} records")
print(f"📋 Fields: {list(df.columns)}")
df.head()

### Let's spot the duplicates manually first

Looking at the data, can you see the potential duplicates? This is exactly what RowVoi helps automate!

In [ ]:
# Let's look for obvious duplicates by examining similar names
print("🔍 Records with similar names:")
print("\nJohn Smith entries:")
print(df[df['last_name'] == 'Smith'].to_string())

print("\nSarah Johnson entries:")
print(df[df['last_name'] == 'Johnson'].to_string())

## 2. Find Minimal Distinguishing Fields

**Business Question:** *Which customer fields do we actually need to check to identify duplicates reliably?*

Checking every field is expensive and time-consuming. RowVoi finds the minimal set that distinguishes all customers.

In [ ]:
# Let's say we suspect these customers might be duplicates
suspicious_customers = [0, 1, 2, 4, 6, 9, 11, 13, 15, 17]

print(f"🎯 Analyzing {len(suspicious_customers)} potentially duplicate customers")
print("Customer records to distinguish:")
print(df.loc[suspicious_customers][['first_name', 'last_name', 'email', 'phone', 'company']].to_string())

In [ ]:
# Find the minimal set of fields needed to distinguish these customers
minimal_fields = find_key(df, suspicious_customers, strategy="greedy")

print(f"✅ Minimal distinguishing fields: {minimal_fields}")
print(f"📊 Only need to check {len(minimal_fields)} out of {len(df.columns)} fields!")

# Let's see why these fields work
print("\n🔍 How these fields distinguish customers:")
print(df.loc[suspicious_customers][minimal_fields].to_string())

## 3. Cost-Aware Field Selection

**Business Reality:** Some fields are expensive to verify:
- 📧 Email: Easy (already have)
- 📞 Phone: Moderate (might need to call)
- 🏠 Address: Expensive (requires address validation service)
- 🏢 Company: Very expensive (requires manual research)

Let's find the most cost-effective approach:

In [ ]:
# Define realistic costs for verifying each field
field_costs = {
    'first_name': 0.10,     # Very cheap - already have
    'last_name': 0.10,      # Very cheap - already have
    'email': 0.25,          # Cheap - easy to verify
    'phone': 1.00,          # Moderate - might need to call
    'address': 2.50,        # Expensive - address validation API
    'city': 0.50,           # Moderate - part of address
    'state': 0.25,          # Cheap - standardized
    'zip_code': 0.50,       # Moderate - validation needed
    'company': 5.00,        # Very expensive - manual research
    'registration_date': 0.10  # Very cheap - already have
}

print("💰 Field verification costs:")
for field, cost in field_costs.items():
    print(f"  {field}: ${cost:.2f}")

In [ ]:
# Find the most cost-effective distinguishing fields
cost_effective_fields = find_key(df, suspicious_customers, costs=field_costs, strategy="greedy")

# Calculate total costs
minimal_cost = sum(field_costs[field] for field in minimal_fields)
cost_effective_cost = sum(field_costs[field] for field in cost_effective_fields)

print("📊 Cost Comparison:")
print(f"  Minimal fields: {minimal_fields}")
print(f"  Cost: ${minimal_cost:.2f} per customer")
print(f"  \n  Cost-effective fields: {cost_effective_fields}")
print(f"  Cost: ${cost_effective_cost:.2f} per customer")
print(f"  \n💡 Savings: ${(minimal_cost - cost_effective_cost):.2f} per customer")

## 4. Interactive Customer Resolution

**Real-World Scenario:** You're processing customer records and find potential duplicates. Instead of checking all fields, RowVoi guides you to ask the most informative questions first.

Let's simulate resolving a duplicate case step-by-step:

In [ ]:
# Simulate: We found these customers that might be the same person
duplicate_candidates = [0, 1]  # John Smith records that look similar

print("🚨 Potential duplicate detected!")
print("These customers might be the same person:")
print(df.loc[duplicate_candidates][['first_name', 'last_name', 'email', 'phone', 'company']].to_string())

# Create a policy that considers costs
policy = GreedyCoveragePolicy(costs=field_costs, objective="entropy")

# Start an interactive session
session = DisambiguationSession(
    df,
    duplicate_candidates,
    policy=policy,
    feature_costs=field_costs
)

print(f"\n🎯 Starting disambiguation with {len(duplicate_candidates)} candidates")
print(f"📊 Initial uncertainty: {session.state.entropy:.2f} bits")

In [ ]:
# Get the first recommended field to check
suggestion = session.next_question()
print(f"\n💡 RowVoi recommends checking: '{suggestion.col}'")
print(f"   Expected value: {suggestion.score:.3f}")
print(f"   Cost: ${suggestion.marginal_cost:.2f}")

# Show the values for this field
print(f"\n📋 Values for '{suggestion.col}':")
for idx in duplicate_candidates:
    value = df.loc[idx, suggestion.col]
    print(f"   Customer {idx}: {value}")

In [ ]:
# Simulate checking the recommended field
# Let's say we're trying to verify customer 0 (first John Smith)
true_customer = duplicate_candidates[0]
observed_value = df.loc[true_customer, suggestion.col]

# Record our observation
step = session.observe(suggestion.col, observed_value)

print(f"✅ Checked '{suggestion.col}' = '{observed_value}'")
print(f"📊 Uncertainty reduced: {step.entropy_before:.2f} → {step.entropy_after:.2f} bits")
print(f"💰 Cost: ${step.cost:.2f}")
print(f"\n🎯 Remaining candidates: {len(session.state.candidate_rows)}")
print(f"🏆 Unique identification: {session.state.is_unique}")

## 5. Measure Business Impact

Let's calculate the real business value of this deduplication strategy:

In [ ]:
# Business impact calculation
total_customers = len(df)
estimated_duplicates = 8  # Based on our manual inspection
duplicate_rate = estimated_duplicates / total_customers

# Costs of NOT deduplicating
marketing_cost_per_customer = 25.00  # Cost per marketing campaign per customer
campaigns_per_year = 12
wasted_marketing = estimated_duplicates * marketing_cost_per_customer * campaigns_per_year

# Costs of our deduplication approach
verification_cost_per_customer = cost_effective_cost
deduplication_cost = suspicious_customers.__len__() * verification_cost_per_customer

print("📊 BUSINESS IMPACT ANALYSIS")
print("=" * 40)
print(f"📈 Total customers: {total_customers:,}")
print(f"🔍 Estimated duplicates: {estimated_duplicates}")
print(f"📊 Duplication rate: {duplicate_rate:.1%}")
print()
print(f"💸 Annual cost of duplicates: ${wasted_marketing:,.2f}")
print(f"💰 One-time deduplication cost: ${deduplication_cost:.2f}")
print(f"🏆 Net annual savings: ${wasted_marketing - deduplication_cost:,.2f}")
print(f"📈 ROI: {((wasted_marketing - deduplication_cost) / deduplication_cost * 100):.0f}%")

## 6. Production Implementation Strategy

Based on our analysis, here's your recommended approach:

In [ ]:
print("🚀 PRODUCTION STRATEGY")
print("=" * 30)
print(f"\n1️⃣ Primary deduplication fields: {cost_effective_fields}")
print(f"   💰 Cost per verification: ${cost_effective_cost:.2f}")

print("\n2️⃣ Automated workflow:")
print("   • Run weekly scans for potential duplicates")
print("   • Use RowVoi to prioritize which records to investigate")
print("   • Focus on high-value customers first")

print("\n3️⃣ Expected results:")
print(f"   • Eliminate ~{estimated_duplicates} duplicate records")
print(f"   • Save ${wasted_marketing:,.2f} annually in wasted marketing")
print("   • Improve customer experience (no duplicate emails)")
print("   • Get accurate customer metrics")

print("\n💡 Next steps:")
print("   1. Implement this strategy on your full customer database")
print("   2. Set up monitoring for new duplicates")
print("   3. Train your team on the interactive resolution process")
print("   4. Measure and track your deduplication success metrics")

## Key Takeaways

🎯 **Smart Field Selection**: RowVoi identified that you only need to check 1-2 key fields instead of all 10 customer attributes.

💰 **Cost Optimization**: By considering verification costs, you can save money while maintaining accuracy.

🔍 **Interactive Resolution**: For ambiguous cases, RowVoi guides you to ask the most informative questions first.

📊 **Measurable Impact**: Clear ROI calculation shows the business value of systematic deduplication.

---

**Ready for more?** Check out other examples:
- `business_use_cases/survey_optimization.py` - Optimize questionnaire design
- `advanced_algorithms/` - Deep dive into algorithmic techniques